In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## https://www.tensorflow.org/tutorials/keras/regression?hl=ko

###### Preprocessing Data
# Reading Data

data1 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/043_200421_Santafe.csv'
data2 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/045_200422_Santafe.csv'
data3 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/046_200423_Santafe.csv'
data4 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/047_200424_Santafe.csv'

dataFrame_raw = pd.concat(map(pd.read_csv, [data1, data2, data3, data4]), ignore_index=True)

output_variable_names = [
    'CAL_CO2Flowrate_gphr'
    #'CAL_NOxFlowrateLNTOutletValid_gps'
]

variable_names_to_extract = [ 
#   'AUX_NOxLNInlet_ppm', 
#   'AUX_NOxLNTOutlet_ppm',
#   'CAL_Time_sec', 
#   'AUX_RTCDate_YYYYMMDD', 
    'OBD_CalEngLoad_perc',
#   'OBD_EngCoolantTemp_degC', 
    'OBD_EngineSpeed_rpm', 
#   'OBD_VehicleSpeed_kmph', 
#   'OBD_ActEngPerTorque_perc',
#   'OBD_EngRefTorque_Nm', 
#   'OBD_MAFSensor_gps', 
#   'OBD_EGT11_degC', 
#   'OBD_EGT12_degC',
#   'OBD_DPFDiffPress_kPa', 
#   'AUX_NOxSCROutlet_ppm', 
#   'AUX_lambdaLNTInlet_none', 
#   'AUX_lambdaLNTOutlet_none',
#   'AUX_lambdaSCROutlet_none', 
#   'AUX_O2FracLNTInlet_volPerc', 
#   'AUX_O2FracLNTOutlet_volPerc',
#   'AUX_O2FracSCROutlet_volPerc',
#   'AUX_SensAmbTemp_degC',
#   'AUX_SensAmbRH_perc',
#   'AUX_SensBaroPress_kPa',
#   'AUX_SensTempTurbinOutlet_degC',
#   'AUX_SensTempInManiInlet_degC',
#   'AUX_SensTempTurbinInlet_degC',
#   'AUX_SensTempLNTOutlet_degC',
#   'AUX_SensTempSCROutlet_degC',
#   'AUX_SensTempCompressorOutlet_degC',
#   'AUX_SensPressEGRCoolerInlet_absBar',
#   'AUX_SensDiffPressDPF_kPa',
#   'AUX_SensDiffPressSCR_kPa',
#   'CAL_ExhFlowrate_gps',
    'CAL_CO2Flowrate_gphr',
#   'CAL_NOxFlowrateLNTInletValid_gps',
#   'CAL_NOxFlowrateLNTOutletValid_gps',
#   'CAL_NOxFlowrateSCROutletValid_gps'
]

dataFrame = dataFrame_raw[[
    'OBD_CalEngLoad_perc',
    #'OBD_EngCoolantTemp_degC', 
    'OBD_EngineSpeed_rpm', 
    'CAL_CO2Flowrate_gphr'
]]

#print(dataFrame.head())

###### Cleaning data
RPM_min = 100                      # excluding engine-stop
NOx_max = 1649                     # excluding clipped NOx data (since the sensor maxed out at 1650 ppm)

dataFrame = dataFrame.loc[(dataFrame['OBD_EngineSpeed_rpm'] >= RPM_min)] #& (dataFrame['AUX_NOxLNInlet_ppm'] <= NOx_max))]
#dataFrame = dataFrame.drop(['AUX_NOxLNInlet_ppm'], axis=1)
dataFrame = dataFrame.dropna() 

train_dataset = dataFrame.sample(frac=0.8,random_state=0)
test_dataset = dataFrame.drop(train_dataset.index)
#print(train_dataset.tail())


###### Split train set & test set
train_labels = train_dataset.pop('CAL_CO2Flowrate_gphr')
test_labels = test_dataset.pop('CAL_CO2Flowrate_gphr')
#print(train_labels.head())


In [ ]:
# define the model
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(10, input_dim=2, kernel_initializer='normal', activation='relu'))
	model.add(Dense(5, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, train_dataset, train_labels, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))